# Enrollment_and_Forecast_History_plots

E. Quinn 5/4/2021

Plots of enrollment data and projections from various sources

### Import standard python datascience packages

In [ ]:
import sys
import math
import re
import copy as cp
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
#plt.switch_backend('WebAgg')
import seaborn as sns
import pickle
%matplotlib inline

In [ ]:
from datetime import datetime, timedelta, date
from datascience import *
import uuid
import random

In [ ]:
from scipy.stats import nbinom
from scipy.stats import poisson
from scipy.stats import binom
from scipy.stats import betabinom
import numpy.random as npr

### Set path to data files

In [ ]:
data_path = '../'
!pwd

In [ ]:
with open('../Enrollment_and_Forecast_History_5_3_2021.pkl', 'rb') as handle:
    edict = pickle.load(handle)

In [ ]:
#edict

In [ ]:
pdata = {}


for year in edict.keys():
    for grade in edict[year]['grades'].keys():
        if grade not in pdata.keys():
            pdata[grade] = {}
        for Type in edict[year]['grades'][grade].keys():
            if Type not in pdata[grade].keys():
                pdata[grade][Type] = {}
            for source in edict[year]['grades'][grade][Type].keys():
                if source not in pdata[grade][Type].keys():
                    pdata[grade][Type][source] = {}
                pdata[grade][Type][source][year] = edict[year]['grades'][grade][Type][source]
            
#pdata

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def set_data(pdata,ddict,Type,source,grade):
    if grade not in ddict.keys():
        ddict[grade] = {}
    if Type not in ddict[grade].keys():
        ddict[grade][Type] = {}
    if source not in ddict[grade][Type].keys():
        ddict[grade][Type][source] = {}
    ddict[grade][Type][source]['x'] = make_array()
    ddict[grade][Type][source]['y'] = make_array()
    for year in pdata[grade][Type][source].keys():
        ddict[grade][Type][source]['x'] = np.append(ddict[grade][Type][source]['x'],year)
        ddict[grade][Type][source]['y'] = np.append(ddict[grade][Type][source]['y'],pdata[grade][Type][source][year])
    return(ddict)

ddict = {}

for grade_no in np.arange(1,12+1):
    grade = str(grade_no)
    set_data(pdata,ddict,'forecast','NESD2021',grade)
    set_data(pdata,ddict,'forecast','NESD1819',grade)
    set_data(pdata,ddict,'actual','NESD2021',grade)
    set_data(pdata,ddict,'actual','NESD1819',grade)
    set_data(pdata,ddict,'actual','RIDE_Oct',grade)
    set_data(pdata,ddict,'forecast','M_and_M',grade)
    set_data(pdata,ddict,'actual','M_and_M',grade)
    set_data(pdata,ddict,'forecast','M_and_M_low',grade)
    set_data(pdata,ddict,'forecast','M_and_M_high',grade)

In [ ]:
Colors = {'RIDE_Oct':'black','NESD1819':'limegreen','NESD2021':'turquoise',\
              'M_and_M':'orangered','M_and_M_low':'firebrick','M_and_M_high':'lightcoral'}

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_grade(ddict,grade,colors):
    
    ax = plt.axes([0, 0, 1.78, 1.78])
    
    for Type in ddict[grade].keys():
        for source in ddict[grade][Type].keys():
            if (source != 'RIDE_Oct'):
                if (Type == 'forecast'):
                    plt.plot(ddict[grade][Type][source]['x'],\
                        ddict[grade][Type][source]['y'],\
                        label=source + ' ' + Type,linestyle='--',color=colors[source])
                else:
                    plt.plot(ddict[grade][Type][source]['x'],\
                        ddict[grade][Type][source]['y'],\
                        label=source + ' ' + Type,linestyle='-',color=colors[source])
            if 'RIDE_Oct' in ddict[grade][Type].keys():
                source = 'RIDE_Oct'
                plt.plot(ddict[grade][Type][source]['x'],\
                    ddict[grade][Type][source]['y'],\
                    label=source + ' ' + Type,linestyle='-',color=colors[source])
    plt.legend(loc="lower left")
    
    ax.set_xlabel('Year', fontsize = 18)
    ax.set_ylabel('Enrollment', fontsize = 18)
    ax.set_title('Enrollment and Forecast History - Grade ' + grade, fontsize = 20)

    plt.show()
    return

In [ ]:
for grade in ddict.keys():
    plot_grade(ddict,grade,Colors)